In [ ]:
# pip install openpyxl pandas

In [22]:
import openpyxl
import pandas as pd
from openpyxl.utils import range_boundaries

# Load the workbook
wb = openpyxl.load_workbook('your_file.xlsx', data_only=False)  # Ensure formulas are not evaluated

# Create a list to store the data
data = []

# Loop through each sheet
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]
    
    # Get the header names (first row of the sheet)
    headers = [str(cell.value) if cell.value is not None else "Empty" for cell in ws[1]]  # Convert headers to strings
    
    # Get data validation rules for the sheet
    dv_ranges = []
    for dv in ws.data_validations.dataValidation:
        # Extract the ranges where data validation is applied
        for range_str in dv.sqref.ranges:
            dv_ranges.append(range_str)
    
    # Loop through each column
    for col_idx, col in enumerate(ws.iter_cols(), start=1):
        # Get the column name from the headers list
        column_name = headers[col_idx - 1] if col_idx - 1 < len(headers) else f"Column {col_idx}"
        
        # Get the first row example (value from the second row, assuming row 1 is headers)
        first_row_cell = ws.cell(row=2, column=col_idx)
        if isinstance(first_row_cell.value, openpyxl.worksheet.formula.ArrayFormula):
            first_row_example = first_row_cell.value.text  # Extract the formula string
        elif first_row_cell.value and str(first_row_cell.value).startswith('='):
            first_row_example = str(first_row_cell.value)  # Use the formula itself
        else:
            first_row_example = str(first_row_cell.value) if first_row_cell.value is not None else "Empty"  # Convert to string
        
        # Loop through each cell in the column
        for cell in col:
            # Check if the cell has a formula
            has_formula = cell.value and str(cell.value).startswith('=')
            
            # Check if the cell is within any data validation range
            has_data_validation = False
            cell_coord = cell.coordinate
            for range_str in dv_ranges:
                try:
                    min_col, min_row, max_col, max_row = range_boundaries(range_str)
                    if (min_col <= cell.column <= max_col and
                        min_row <= cell.row <= max_row):
                        has_data_validation = True
                        break
                except (ValueError, TypeError):
                    # Skip invalid range strings
                    continue
            
            # Handle formulas
            if isinstance(cell.value, openpyxl.worksheet.formula.ArrayFormula):
                formula_value = cell.value.text  # Extract the formula string
            elif has_formula:
                formula_value = str(cell.value)  # Extract the formula string
            else:
                formula_value = None  # No formula
            
            # Append data for all cells
            data.append({
                'Sheet Name': sheet_name,
                'Column Name': column_name,
                'First Row Example': first_row_example,  # Value from the first row of the column
                'Has Formula or Data Validation': has_formula or has_data_validation,
                'Formula': formula_value  # Handle formulas
            })

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Group by 'Sheet Name', 'Column Name', and 'First Row Example'
grouped = df.groupby(['Sheet Name', 'Column Name', 'First Row Example'], dropna=False).agg({
    'Has Formula or Data Validation': 'any',  # If any row in the group has True, set to True
    'Formula': lambda x: ', '.join(set([str(val) for val in x if val is not None]))  # Combine unique formulas
}).reset_index()

# Export to CSV
grouped.to_csv('sheet_info.csv', index=False)

print("Sheet information exported successfully to 'sheet_info.csv'!")

Sheet information exported successfully to 'sheet_info.csv'!


/opt/homebrew/Caskroom/miniconda/base/envs/dev/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
